In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)

urlPlayer <- "https://raw.githubusercontent.com/aric-chan/dsci-100-2024w2-project-005-6/0dbc21753a915e3c1f35c50505f65622deecdc0d/players.csv"
urlSessions <- "https://raw.githubusercontent.com/aric-chan/dsci-100-2024w2-project-005-6/refs/heads/main/sessions.csv"
player_data <- read_csv(urlPlayer)
sessions_data <- read_csv(urlSessions)
# View the first few rows
head(player_data)
head(sessions_data)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


In [44]:
playerData<- player_data |>
            mutate(age = Age)|>
              mutate(age_group = case_when(
                age <= 16 ~ "Teen",     # Group 1: Age 25 and below
                age > 16 & age <= 23 ~ "young adult",  # Group 2: Age 26 to 50
                age > 23 ~ "Adult"       # Group 3: Age above 50
              ))|>
            select(experience, age, age_group, played_hours)
head(playerData)
unique(playerData$experience)
# Convert to ordered factor
experience_ord <- factor(playerData$experience, 
                           levels = c("Beginner", "Regular", "Amateur", "Veteran", "Pro"),  
                           ordered = TRUE)
playerData$experience_num <- as.numeric(experience_ord)

age_ord <- factor(playerData$age_group, 
                           levels = c("Teen", "young adult", "Adult"),  
                           ordered = TRUE)
playerData$age_num <- factor(age_ord, ordered = TRUE)


head(playerData)

experience,age,age_group,played_hours
<chr>,<dbl>,<chr>,<dbl>
Pro,9,Teen,30.3
Veteran,17,young adult,3.8
Veteran,17,young adult,0.0
Amateur,21,young adult,0.7
Regular,21,young adult,0.1
Amateur,17,young adult,0.0


[1] "Pro"      "Veteran"  "Amateur"  "Regular"  "Beginner"

experience,age,age_group,played_hours,experience_num,age_num
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<ord>
Pro,9,Teen,30.3,5,Teen
Veteran,17,young adult,3.8,4,young adult
Veteran,17,young adult,0.0,4,young adult
Amateur,21,young adult,0.7,3,young adult
Regular,21,young adult,0.1,2,young adult
Amateur,17,young adult,0.0,3,young adult


In [45]:
#split training test
set.seed(123)  # Ensure reproducibility
data_split <- initial_split(playerData, prop = 0.8)  # 70% training, 30% testing
train_data <- training(data_split)
test_data  <- testing(data_split)

In [51]:
# 1. Create the recipe
reg_recipe <- recipe(played_hours ~ age_num + experience_num, data = train_data)

In [52]:
reg_model <- linear_reg() %>%
  set_engine("lm")  # Use base R's lm() function


In [53]:
reg_workflow <- workflow() %>%
  add_recipe(reg_recipe) %>%
  add_model(reg_model)


In [54]:
fitted_workflow <- fit(reg_workflow, data = train_data)

In [55]:
predictions <- predict(fitted_workflow, new_data = test_data) %>%
  bind_cols(test_data)  # Combine with actual values

In [56]:
metrics <- predictions %>%
  metrics(truth = played_hours, estimate = .pred)

print(metrics)  # Show RMSE, R-squared, and MAE

# A tibble: 3 × 3
  .metric .estimator .estimate
  <chr>   <chr>          <dbl>
1 rmse    standard    10.4    
2 rsq     standard     0.00968
3 mae     standard     7.38   
